In [1]:
import pandas as pd
import numpy as np

In [2]:
dataDir = "data/"  # Directory where our data is stored

In [3]:
dtypes = {'display_id' : np.int32, 'ad_id': np.int32, 'clicked': np.bool}
dfTrain = pd.read_csv(dataDir + "clicks_train.csv", dtype=dtypes)

In [4]:
dfTrain.head()

,display_id,ad_id,clicked
0,1,42337,False
1,1,139684,False
2,1,144739,True
3,1,156824,False
4,1,279295,False


In [5]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87141731 entries, 0 to 87141730
Data columns (total 3 columns):
display_id    int32
ad_id         int32
clicked       bool
dtypes: bool(1), int32(2)
memory usage: 747.9 MB


In [6]:
dfClicked = dfTrain[dfTrain['clicked'] == 1]

In [7]:
dfClicked.head()

,display_id,ad_id,clicked
2,1,144739,True
9,2,308455,True
14,3,228657,True
17,4,153623,True
25,5,326514,True


In [8]:
dfClicked = dfClicked.groupby('ad_id')['clicked'].count().to_frame().reset_index()

In [9]:
dfClicked.head()

,ad_id,clicked
0,2,1
1,3,11
2,4,4
3,7,985
4,8,4


In [10]:
dfOccured = dfTrain.groupby('ad_id')['clicked'].count().to_frame().reset_index()

In [11]:
dfOccured.head()

,ad_id,clicked
0,1,2
1,2,22
2,3,161
3,4,32
4,5,1


In [12]:
dfNew = dfClicked.merge(dfOccured, how='left', on='ad_id')

In [13]:
dfNew = dfNew.rename(columns={'clicked_x':'clicked', 'clicked_y': 'viewed'})

In [14]:
#dfNew['likelihood'] = dfNew['clicked'] / dfNew['viewed']
dfNew['likelihood'] = dfNew['clicked']

In [15]:
#dfNew.to_csv(dataDir + 'clickonly.csv', index=False)

# Now try with the test file

In [16]:
dfNew = dfNew.drop(['clicked', 'viewed'], axis=1)

In [17]:
dfTest = pd.read_csv(dataDir + "clicks_test.csv")

In [18]:
dfTest = dfTest.merge(dfNew, how='left')

In [19]:
dfTest.head()

,display_id,ad_id,likelihood
0,16874594,66758,373.0
1,16874594,150083,5261.0
2,16874594,162754,15919.0
3,16874594,170392,5261.0
4,16874594,172888,5162.0


In [20]:
dfTest['likelihood'].fillna(0, inplace=True)

In [21]:
dfTest.sort_values(['display_id','likelihood'], inplace=True, ascending=False)

## Create the submission file

In [22]:
subm = dfTest.groupby('display_id')['ad_id'].apply(lambda x: " ".join(map(str,x))).reset_index()

In [23]:
subm.to_csv(dataDir + "submissionCutoutNonClicked.csv.gz", header=True, index=False, compression='gzip')